In [ ]:
import numpy as np

In [ ]:
class SGD():
    # Stochastic Gradient Descent
    
    def __init__(
        self, lr=0.01, moentum=0.0, clip_norm=None, lr_scheduler=None, **kwargs
    ):
        r'''
        momentum : float in range [0, 1]
            动量
        clip_norm : float
            梯度裁剪，将梯度变化限制在某一范围内，避免梯度爆炸。
            https://zhuanlan.zhihu.com/p/99953668（下面使用的于此不同）
        lr_scheduler : str
            学习率调整器，输入调整器文件包。如无调整器，则使用常数lr作为学习率。
        '''
        super().__init__(lr, lr_scheduler)
        self.hyperparameters = {
            'id': 'SGD',
            'lr': lr,
            'momentum': momentum,
            'clip_norm': clip_norm,
            'lr_scheduler': str(self.lr_scheduler),
        }
        
    def __str__(self):
        H = self.hyperparameters
        lr, mm, cn, ls = H['lr'], H['momentum'], H['clip_norm'], H['lr_scheduler']
        return 'SGD(lr={}, momentum={}, clip_norm={}, lr_scheduler={})'.format(
            lr, mm, cn, ls)
    
    def  update(self, param, param_grad, param_name='omega', cur_loss=None):
        r'''
        param : <numpy.ndarray>` of shape (n, m)
        param_grad : <numpy.ndarray>` of shape (n, m)
            计算后的梯度矩阵。
        param_name : str
            参数历史梯度保存在缓存中所使用的名字，默认为omega。
        cur_loss : float
            用于调整学习率调整器？？？
        '''
        C = self.cache
        H = self.hyperparameters
        momentum, clip_norm = H['momentum'], H['clip_norm']
        lr = self.lr_scheduler(self.cur_step, cur_loss)
        # 学习率调整？？？
        
        if param_name not in C:
            C[param_name] = np.zeros_like(param_grad)# 创建梯度记录
            
        t = np.inf if clip_norm is None else clip_norm
        if norm(param_grad) > t:# 梯度裁剪过程
            param_grad = param_grad * t / norm(param_grad)
            
        update = momentum * C[param_name] + lr * param_grad# 带动量的随机梯度下降
        self.cache[param_name] = update # 更新历史梯度，决定下一次动量的大小、方向
        return param - update